In [5]:
# use the model in Model, and dataloader in Dataloaders
# to train the model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from Models import mobilenetv2
from Dataloaders import dataloader_cifar10
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import Config.external_paths as external_paths
# get the dataset
train, test, labels = dataloader_cifar10.Dataloader_cifar10(train_batch=128, test_batch=100, seed=2024)

# 2. transfer the dataset to fit the model, for the training, client and server model are all on the server
client_model, server_model = mobilenetv2.stupid_model_splitter(weight_path='./Weights/cifar-10/MobileNetV2.pth')

# gating = some_gating_function()

# reducer = some_reducer_funtion()

# generator = some_generator_funtion()

client_model = client_model.cuda()

# infer the model
client_model.eval()

# create a file path
embeddings_path = external_paths.tintin_path + 'embeddings/cifar-10/MobileNetV2/'
embedding_tensor = None
labels_tensor = None
# the embedding of the model and store
if not os.path.exists(embeddings_path):
    os.makedirs(embeddings_path)
if not os.path.exists(embeddings_path+ 'embeddings/'):
    os.makedirs(embeddings_path+ 'embeddings/')
if not os.path.exists(embeddings_path+ 'labels/'):
    os.makedirs(embeddings_path+ 'labels/')
with torch.no_grad():
    for ind, data in tqdm(enumerate(train)):
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = client_model(inputs)

        outputs_cpu = outputs.cpu()
        # if embedding_tensor is None:
        #     embedding_tensor = outputs_cpu
        #     labels_tensor = labels
        # else:
        #     embedding_tensor = torch.cat((embedding_tensor, outputs_cpu), dim=0)
        #     labels_tensor = torch.cat((labels_tensor, labels), dim=0)
        embedding_tensor = outputs_cpu
        labels_tensor = labels
        
        # store the embedding

        torch.save(embedding_tensor, embeddings_path + 'embeddings/' + str(ind) + '.pth')
        torch.save(labels_tensor, embeddings_path + 'labels/' + str(ind) + '.pth')

    

Files already downloaded and verified
Files already downloaded and verified


23it [00:03,  6.68it/s]


KeyboardInterrupt: 

In [2]:
# this part of code is for getting the embeddings of the server output 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from Models import mobilenetv2
from Dataloaders import dataloader_cifar10
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
# get the dataset
train, test, labels = dataloader_cifar10.Dataloader_cifar10(train_batch=128, test_batch=100, seed=2024)

# 2. transfer the dataset to fit the model, for the training, client and server model are all on the server
client_model, server_model = mobilenetv2.stupid_model_splitter(weight_path='./Weights/cifar-10/MobileNetV2.pth')

# gating = some_gating_function()

# reducer = some_reducer_funtion()

# generator = some_generator_funtion()

client_model = client_model.cuda()
server_model = server_model.cuda()

# infer the model
client_model.eval()
server_model.eval()

# create a file path
embeddings_path = '../data/cifar-10-embedding-out/'
embedding_tensor = None
labels_tensor = None
# the embedding of the model and store
if not os.path.exists(embeddings_path):
    os.makedirs(embeddings_path)
if not os.path.exists(embeddings_path+ 'embeddings/'):
    os.makedirs(embeddings_path+ 'embeddings/')
with torch.no_grad():
    for ind, data in tqdm(enumerate(train)):
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = client_model(inputs)

        # outputs_cpu = outputs.cpu()
        # # if embedding_tensor is None:
        # #     embedding_tensor = outputs_cpu
        # #     labels_tensor = labels
        # # else:
        # #     embedding_tensor = torch.cat((embedding_tensor, outputs_cpu), dim=0)
        # #     labels_tensor = torch.cat((labels_tensor, labels), dim=0)
        # embedding_tensor = outputs_cpu
        # labels_tensor = labels
        outputs = server_model(outputs)
        
        # store the embedding

        torch.save(outputs, embeddings_path + 'embeddings/' + str(ind) + '.pth')
        # torch.save(labels_tensor, embeddings_path + 'labels-out/' + str(ind) + '.pth')

    

Files already downloaded and verified
Files already downloaded and verified


391it [00:02, 155.20it/s]


In [13]:
# this part of code is for getting transferred embeddings of the client model
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from Models import mobilenetv2
from Dataloaders import dataloader_cifar10
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
from tqdm import tqdm
import scipy
# get the dataset

def calculate_entropy(embs, percentage):
    # embs are torch tensors
    # percentage shows the number of embeddings to select
    embs_entropy = []
    for i in range(embs.size(len(embs.size())-3)):
        embs_entropy.append(scipy.stats.entropy(embs[:, i].reshape(-1))) # b, c
    # get the top percentage of the channels
    embs_entropy = np.array(embs_entropy)
    indices = np.argsort(embs_entropy)
    num_selected = int(embs.size(1) * percentage)
    selected_indices = indices[:num_selected]
    return selected_indices
    # out b, c*p, h, w or c*p, h, w

def ranker_entropy(embs, percentage):
    # calculate the entropy of the embeddings
    selected_indices = calculate_entropy(embs, percentage)
    # select indice from test embeddings
    selected_embeddings = embs[:, selected_indices] # b, c*p, h, w
    return selected_embeddings, selected_indices


train, test, labels = dataloader_cifar10.Dataloader_cifar10(train_batch=128, test_batch=100, seed=2024)

# 2. transfer the dataset to fit the model, for the training, client and server model are all on the server
client_model, server_model = mobilenetv2.stupid_model_splitter(weight_path='./Weights/cifar-10/MobileNetV2.pth')

rankers = []
rank_rate = [0.25, 0.5, 0.75]
for rate in rank_rate:
    rankers.append(ranker_entropy)
# gating = some_gating_function()

# reducer = some_reducer_funtion()

# generator = some_generator_funtion()

client_model = client_model.cuda()
server_model = server_model.cuda()

# infer the model
client_model.eval()
server_model.eval()

# create a file path
embeddings_paths = ['../data/cifar-10-embedding-entropy/' + str(x) + '/' for x in range(3)]
# create path
for embeddings_path in embeddings_paths:
    if not os.path.exists(embeddings_path):
        os.makedirs(embeddings_path)
    if not os.path.exists(embeddings_path+ 'embeddings/'):
        os.makedirs(embeddings_path+ 'embeddings/')

embedding_tensor = None
labels_tensor = None
# the embedding of the model and store
for embeddings_path in embeddings_paths:
    if not os.path.exists(embeddings_path):
        os.makedirs(embeddings_path)
    if not os.path.exists(embeddings_path+ 'embeddings/'):
        os.makedirs(embeddings_path+ 'embeddings/')
with torch.no_grad():
    for ind, data in tqdm(enumerate(train)):
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = client_model(inputs)

        # outputs_cpu = outputs.cpu()
        # # if embedding_tensor is None:
        # #     embedding_tensor = outputs_cpu
        # #     labels_tensor = labels
        # # else:
        # #     embedding_tensor = torch.cat((embedding_tensor, outputs_cpu), dim=0)
        # #     labels_tensor = torch.cat((labels_tensor, labels), dim=0)
        # embedding_tensor = outputs_cpu
        # labels_tensor = labels

        # add rankers here
        outputs = outputs.cpu()
        emb1 = rankers[0](outputs, 0.25)
        emb2 = rankers[1](outputs, 0.5)
        emb3 = rankers[2](outputs, 0.75)
        
        # store the embedding
        torch.save(emb1, embeddings_paths[0] + 'embeddings/' + str(ind) + '.pth')
        torch.save(emb2, embeddings_paths[1] + 'embeddings/' + str(ind) + '.pth')
        torch.save(emb3, embeddings_paths[2] + 'embeddings/' + str(ind) + '.pth')

        # torch.save(labels_tensor, embeddings_path + 'labels-out/' + str(ind) + '.pth')

    

Files already downloaded and verified
Files already downloaded and verified


0it [00:00, ?it/s]

torch.Size([128, 8, 32, 32])


In [11]:
# test ranker embedding
p = '/home/tonypeng/Workspace1/adaptfilter/data/cifar-10-embedding-entropy/1/embeddings/0.pth'
embs = torch.load(p)
print(embs[1])

[24  9  7 27 31 28 13 14 25  4 23  5 26 18 17 10]
